In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.2uiHo3Wxqh/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.eEhinfWy8p/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.MAp8MqVZtw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
!pip install ipykernel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00


In [ ]:
import ipykernel
import os
import socket

def limit_io_pub():
    max_bytes = 100000000  # maximum 100MB/s
    connection_file = ipykernel.get_connection_file()
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]
    iopub_socket = socket.SocketType(socket.AF_UNIX)
    iopub_socket.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, max_bytes)
    iopub_socket.connect(f"{os.path.dirname(connection_file)}/kernel-{kernel_id}/iopub.sock")
    os.environ["IOPUB_DATA_RATE_LIMIT"] = str(max_bytes)


In [ ]:
from selenium import webdriver

In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
a = []

In [ ]:
a

[]

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
def scrape(url):
    driver = web_driver()
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all("div", class_="searchstyles__ItemRow-ci5zlg-4 jblhXc")
    for article in articles:
      title = article.find('h6', class_='typography__StyledTypography-owin6q-0 hjHKEC')
      link = article.a["href"]
      link = "https://www.coindesk.com" + link
      date = article.find('h6' , class_= 'typography__StyledTypography-owin6q-0 lfNAOh')
      if date is not None:
        date = date.get_text()
      if title is not None:
        title = title.get_text()
      row = (title,date,link)
      a.append(row)

In [ ]:
for i in range(300):
  page_url = "https://www.coindesk.com/search?s=bitcoin&sort=1&i=" + str(i)
  print(page_url)
  scrape(page_url)

https://www.coindesk.com/search?s=bitcoin&sort=1&i=0
https://www.coindesk.com/search?s=bitcoin&sort=1&i=1
https://www.coindesk.com/search?s=bitcoin&sort=1&i=2
https://www.coindesk.com/search?s=bitcoin&sort=1&i=3
https://www.coindesk.com/search?s=bitcoin&sort=1&i=4
https://www.coindesk.com/search?s=bitcoin&sort=1&i=5
https://www.coindesk.com/search?s=bitcoin&sort=1&i=6
https://www.coindesk.com/search?s=bitcoin&sort=1&i=7
https://www.coindesk.com/search?s=bitcoin&sort=1&i=8
https://www.coindesk.com/search?s=bitcoin&sort=1&i=9
https://www.coindesk.com/search?s=bitcoin&sort=1&i=10
https://www.coindesk.com/search?s=bitcoin&sort=1&i=11
https://www.coindesk.com/search?s=bitcoin&sort=1&i=12
https://www.coindesk.com/search?s=bitcoin&sort=1&i=13
https://www.coindesk.com/search?s=bitcoin&sort=1&i=14
https://www.coindesk.com/search?s=bitcoin&sort=1&i=15
https://www.coindesk.com/search?s=bitcoin&sort=1&i=16
https://www.coindesk.com/search?s=bitcoin&sort=1&i=17
https://www.coindesk.com/search?s=bitc

In [ ]:
df = pd.DataFrame(a, columns=['Title', 'Date','Link'])

In [ ]:
b = []

In [ ]:
df

,Title,Date,Link
0,None,None,https://www.coindesk.com/price/bitcoin
1,Headlines: Top Stories of the Week 05-01-23,"May 6, 2023",https://www.coindesk.com/podcasts/coindesk-pod...
2,New York Attorney General Seeks New Crypto Pow...,"May 6, 2023",https://www.coindesk.com/podcasts/coindesk-pod...
3,"Voyager Digital Plans to Liquidate Assets, Win...","May 5, 2023",https://www.coindesk.com/policy/2023/05/05/voy...
4,Bitcoin Ordinals Controversy Brews; Montana's ...,"May 5, 2023",https://www.coindesk.com/video/bitcoin-ordinal...
...,...,...,...
3294,Elon Musk Tweet Sparks Flurry of Twitter-Theme...,"Nov 1, 2022",https://www.coindesk.com/tech/2022/11/01/elon-...
3295,Dogecoin Led the Pack Among Cryptocurrencies i...,"Nov 1, 2022",https://www.coindesk.com/markets/2022/11/01/do...
3296,First Mover Asia: Hong Kong FinTech Chair Prai...,"Nov 1, 2022",https://www.coindesk.com/markets/2022/11/01/fi...
3297,Market Wrap: Dogecoin’s October Finale Trumps ...,"Oct 31, 2022",https://www.coindesk.com/markets/2022/10/31/ma...


In [ ]:
df.to_csv('1coindesk_scrape.csv', index=False)

In [ ]:
def scrape(url):
    driver = web_driver()
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all("div", class_="searchstyles__ItemRow-ci5zlg-4 jblhXc")
    for article in articles:
      title = article.find('h6', class_='typography__StyledTypography-owin6q-0 hjHKEC')
      link = article.a["href"]
      link = "https://www.coindesk.com" + link
      date = article.find('h6' , class_= 'typography__StyledTypography-owin6q-0 lfNAOh')
      if date is not None:
        date = date.get_text()
      if title is not None:
        title = title.get_text()
      row = (title,date,link)
      b.append(row)

In [ ]:
for i in range(100,200):
  page_url = "https://www.coindesk.com/search?s=bitcoin&sort=1&i=" + str(i)
  print(page_url)
  scrape(page_url)

https://www.coindesk.com/search?s=bitcoin&sort=1&i=300
https://www.coindesk.com/search?s=bitcoin&sort=1&i=301
https://www.coindesk.com/search?s=bitcoin&sort=1&i=302
https://www.coindesk.com/search?s=bitcoin&sort=1&i=303
https://www.coindesk.com/search?s=bitcoin&sort=1&i=304
https://www.coindesk.com/search?s=bitcoin&sort=1&i=305
https://www.coindesk.com/search?s=bitcoin&sort=1&i=306
https://www.coindesk.com/search?s=bitcoin&sort=1&i=307
https://www.coindesk.com/search?s=bitcoin&sort=1&i=308
https://www.coindesk.com/search?s=bitcoin&sort=1&i=309
https://www.coindesk.com/search?s=bitcoin&sort=1&i=310
https://www.coindesk.com/search?s=bitcoin&sort=1&i=311
https://www.coindesk.com/search?s=bitcoin&sort=1&i=312
https://www.coindesk.com/search?s=bitcoin&sort=1&i=313
https://www.coindesk.com/search?s=bitcoin&sort=1&i=314
https://www.coindesk.com/search?s=bitcoin&sort=1&i=315
https://www.coindesk.com/search?s=bitcoin&sort=1&i=316
https://www.coindesk.com/search?s=bitcoin&sort=1&i=317
https://ww

In [ ]:
len(b)

877

In [ ]:
df2 = pd.DataFrame(b, columns=['Title', 'Date','Link'])

In [ ]:
df2

,Title,Date,Link
0,None,None,https://www.coindesk.com/price/bitcoin
1,Weekend Story | Crypto’s Macro Drivers – It’s ...,"May 7, 2023",https://www.coindesk.com/podcasts/markets-dail...
2,Headlines: Top Stories of the Week 05-01-23,"May 6, 2023",https://www.coindesk.com/podcasts/coindesk-pod...
3,New York Attorney General Seeks New Crypto Pow...,"May 6, 2023",https://www.coindesk.com/podcasts/coindesk-pod...
4,"Voyager Digital Plans to Liquidate Assets, Win...","May 5, 2023",https://www.coindesk.com/policy/2023/05/05/voy...
...,...,...,...
872,Goldman's Bullish Stance on 'Real Bond Yield' ...,"Sep 19, 2022",https://www.coindesk.com/markets/2022/09/19/go...
873,Bitcoin Miner Bitfarms Starts Production in Ar...,"Sep 19, 2022",https://www.coindesk.com/business/2022/09/19/b...
874,"As Ether, Bitcoin Wilt, Trading Firms Blame La...","Sep 19, 2022",https://www.coindesk.com/markets/2022/09/19/as...
875,First Mover Asia: Cryptos Slide in Weekend Tra...,"Sep 19, 2022",https://www.coindesk.com/markets/2022/09/19/fi...


In [ ]:
df2.to_csv('part2_coindesk_scrape.csv', index=False)